In [ ]:
#1 what is the approximate depth of a decision tree trained on a training set with one million instances
#depth containing m leaves is equal to log(m)^2. so log(10^6) is about 20

In [ ]:
#2 is a nodes gini impurity generally lower or greater than its parents? is it generally lower/greater, or always lower/greater?
#generally lower than its parents.it is because the cost function is split in a way that minimizes the weighted sum.

In [ ]:
#3 If a decision tree is overfitting the training set, is it a good idea to try decreasing max_depth?
#it is a good idea to decrease max_depth sine this will contrain the model, it regularizes it

In [ ]:
#4 if a decision tree is underfitting the training set, is it a good idea to try scaling the input features?
#it does not matter if it is scaled or not. scaling the data set is a waste of time.

In [ ]:
#5 if it takes one hour to train a decision tree on a training set containing 1 million instances, roughly how much time will it take to train another decision tree on a training set containing 10 million instances
#1 million instances is (n x m x log(10m)) so for 10 million it is (n x 10m x log(10m)) since m = 10^6 the time is about 11.7 hours.

In [7]:
#6 if your training set contains 100,000 instances, will setting presort = True speed up the training?
#presorting the training set speeds up training if the dataset is smaller than a few thousand instances. if over 100,000 it will slow it down tremendously.


In [8]:
#7train and fine tune a decision tree for the moons dataset (a) use make_moons(n_samples=10000, noise=0.4)
#b use train_test_split() c) use grid search with cross validation d)train it on full training set, measure performance.


In [9]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 848 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    1.7s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [12]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [13]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

In [14]:
#8 grow a forest by following the setps:

In [15]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [16]:
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

In [17]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [18]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [19]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872